In [1]:
# 0 Water 
# 1 Strange Water 
# 2 Light Forest 
# 3 Dark Forest 
# 4 Urban 
# 5 Rock 
# 6 Ice 
# 7 Sand
# 8 Thick Clouds 
# 9 Thin Clouds 
# 10 Crop 


#Make labels into land, water and clouds 
#water  = 2
#cropland = 3
#pasture = 4
#city = 5
#rock = 6
#thin_forest = 7
#thick_forest = 8 
#cloud = 9
#thin_clouds = 10

from SVMBDT import *
from HelperFunctions_ import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ground_training_functions import * 
import numpy as np

In [2]:
trondheim_data = plt.imread("../Training_data/Trondheim_2022_08_23T10_26_43-bin3.png")
trondheim_gt = np.load("../Training_data/Trondheim_labels_2022-08-23.npz")["labels"]

#Preprocess the labeling 

new_trondheim_gt = []
y_ = trondheim_gt.flatten()

for i in range(0,len(y_),3): 
    new_trondheim_gt.append(y_[i])

new_trondheim_gt = np.array(new_trondheim_gt)


new_trondheim_gt[new_trondheim_gt == 2] = 0
new_trondheim_gt[new_trondheim_gt == 3] = 10 
new_trondheim_gt[new_trondheim_gt == 8] = 3 
new_trondheim_gt[new_trondheim_gt == 9] = 8 
new_trondheim_gt[new_trondheim_gt == 10] = 9
new_trondheim_gt[new_trondheim_gt == 4] = 10 
new_trondheim_gt[new_trondheim_gt == 7] = 2 
new_trondheim_gt[new_trondheim_gt == 5] = 4 
new_trondheim_gt[new_trondheim_gt == 6] = 5 

trondheim_new_labels = copy.deepcopy(new_trondheim_gt)


store_labeled_image(trondheim_new_labels, len(trondheim_data), len(trondheim_data[0]), "trondheim_test")

In [3]:
def rotate_labels(labeled_image, x_dim, y_dim):

    resized_li = np.array(labeled_image)
    resized_li.resize(y_dim,x_dim)
    rotate_image = []

    for i in range(len(resized_li[0])-1,-1,-1): 
        for j in range(len(resized_li)): 
            rotate_image.append(resized_li[j][i])
    return np.array(rotate_image) 


In [4]:
new_trond = rotate_labels(trondheim_new_labels, len(trondheim_data), len(trondheim_data[0]))

In [1531]:
penguin_data = plt.imread("../Training_data/penguin_2022-12-11_3-bin3.png")
penguin_labels = np.load("../Z_Labeled_HYSPO_Data/penguin.npy")

florida_data = plt.imread("../Training_data/florida_2023-02-12_1515Z-bin3.png")
florida_labels = np.load("../Z_Labeled_HYSPO_Data/florida.npy")

gobabeb_data = plt.imread("../Training_data/gobabeb_2023-02-14_0840Z-bin3.png")
gobabeb_labels = np.load("../Z_Labeled_HYSPO_Data/gobabeb.npy")

kampala_data = plt.imread("../Training_data/kampala_2023-02-04_0740Z-bin3.png") 
kampala_labels = np.load("../Z_Labeled_HYSPO_Data/kampala.npy")

In [1754]:
combined_data_array = [kampala_data, gobabeb_data]
combined_labels_array = [kampala_labels, gobabeb_labels]

In [1551]:
print(len(trondheim_data[0]))
print(len(kampala_data[0]))
print(len(gobabeb_data[0]))

956
956
956


In [1755]:
def combineDataAndLabels(data_array, labels_array): 
    combined_data = []
    combined_labels = []

    for i in range(len(data_array)): 
        for j in range(len(data_array[i])): 
            combined_data.append(data_array[i][j])

    for i in range(len(labels_array)): 
        for j in range(len(labels_array[i])): 
            combined_labels.append(labels_array[i][j])

    return np.array(combined_data), np.array(combined_labels)      

In [1756]:
combined_data, combined_labels = combineDataAndLabels(combined_data_array, combined_labels_array)

In [1757]:
store_labeled_image(combined_labels, len(combined_data), len(combined_data[0]), "test")

In [2256]:
X = np.array(combined_data).reshape((-1,3))
y = np.array(combined_labels).flatten()

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.95, random_state=1)

#tb = {0 : [[6,8,9],[0,2,3,4,5,10]], 1 : [[[8,9],[6]],[[0],[2,3,4,5,10]]]}

#tb = {0 : [[6,7,8,9],[0,2,3,4,5,10]], 1 : [[[6,8,9],[7]],[[0],[2,3,4,5,10]]], 2 : [[[6],[8,9]],[[2,3,4,10],[5]]]}

tb = {0 : [[8],[0,1,2,3,4,5,7,11]], 1 : [[],[[0],[1,2,3,4,5,7,11]]], 2 : [[],[[1],[2,3,4,5,11]]]}

svm_tb = {0 : LinearSVC(), 1 : [False, LinearSVC()], 2 : [False, LinearSVC()]}

svm_branch_models = SvmDesionTreeTrain(X_train, y_train, tb, svm_tb)

The training time is: 0.055 sec.


In [2426]:
svm_branch_models_to_bin(svm_branch_models[0],0.75)
svm_branch_models_to_bin(svm_branch_models[1][1],1.2)
svm_branch_models_to_bin(svm_branch_models[2][1],-0.67)



In [2427]:
test_name = "../Training_data/sittwe_2023-02-07_0344Z-bin3.png"
test_data = plt.imread(test_name)

X_test = np.array(test_data).reshape((-1,3))
#X_test = preprocessing.scale(X_test, axis=0)

In [2428]:
s_bdt = [["0008", "1", "0"],["0001", "0", "2"], ["0102", "0", "0"]]
bdt = bdt_string_to_config_format(s_bdt)
pl, svm_models = bdt_execute(X_test, bdt)

In [2877]:
def fixing_labels(pl, x_0, x_1, y_0, y_1, x_dim, y_dim, old_label, new_label): 
    fixel_label = np.array(pl)
    fixel_label.resize(x_dim, y_dim)

    for i in range(len(fixel_label)): 
        for j in range(len(fixel_label[0])): 
            if((i >= x_0) and i <= x_1): 
                if(j >= y_0 and j <= y_1): 
                    if(fixel_label[i][j] == old_label): 
                        fixel_label[i][j] = new_label

    return fixel_label.flatten()




In [2882]:
store_labeled_image(fpl, len(test_data), len(test_data[0]), "test")

In [2879]:
classesInLabels(fpl)

array([0, 1, 2, 7, 8])

In [2880]:
# 0 Water 
# 1 Strange Water 
# 2 Light Forest 
# 3 Dark Forest 
# 4 Urban 
# 5 Rock 
# 6 Ice 
# 7 Sand
# 8 Thick Clouds 
# 9 Thin Clouds 
# 10 Crop 
# 11 Shadow 